In [1]:
import requests
import json
import openai

openai.api_key = open(".tokens/openai", "r").read()
print("Started")

Started


In [2]:
# Uses anki-connect to connect to anki
response = requests.post('http://localhost:8765', data='{"action": "deckNames", "version": 6}', headers={'Content-type': 'application/json'})
print(response.json())

{'result': ['Driving License', 'English', 'German', 'German::A2_Wortliste_Goethe', 'German::B1_Wortliste_DTZ_Goethe', 'German::Deutsch B1', 'German::Deutsch B1+', 'German::DW Nicos Weg B1', 'Standard'], 'error': None}


In [3]:
def invoke(method, **params):
    request_data = json.dumps({
        'action': method,
        'params': params,
        'version': 6
    })

    response = requests.post('http://localhost:8765', data=request_data)
    response_data = json.loads(response.text)
    print(response_data)

    if response_data.get('error'):
        raise Exception(f"Error: {response_data['error']}")
    else:
        return response_data['result']

def clean_content(markdown: str):
    # this method extracts the code from the markdown and returns only the code from the markdown string

    result = ""
    in_code = False
    for line in markdown.split("\n"):
        if line.startswith("```"):
            in_code = not in_code
        elif in_code:
            result += line + "\n"
    return result

In [19]:
task_template = """
You create cards for Spaced Repetition to for a learner of german as a foreign language, these cards will be used in the context of preparation for the german driving license exam, please use this context if it could help in some cases.
There is an existing structure for cards, for example:

```json
[
    {
        "Wort_DE": "melden",
        "Wort_EN": "to report, register, put one's hand up, volunteer",
        "Verbformen": "meldet, meldete, hat gemeldet",
        "Satz1_DE": "Ich habe schon zweimal angerufen. - Aber es meldet sich niemand.",
        "Satz1_EN": "I have already called twice. - But nobody answers.",
        "Satz2_DE": "Den Unfall müssen Sie der Versicherung melden.",
        "Satz2_EN":"You must report the accident to the insurance company.",
        "Satz3_DE":"Ich melde mich auf Ihre Anzeige. - Sie wollen ein Sofa verkaufen?",
        "Satz3_EN": "I'm reporting to your ad. - You want to sell a sofa?",
        "Satz4_DE":"Warum hast du dich so lange nicht gemeldet? Warst du krank?",
        "Satz4_EN": "Why have not you been in contact for so long? Were you sick?"
    },
    {
        "Wort_DE": "Pflicht",
        "Wort_EN": "Duty; Obligation; Mandatory",
        "Artikel": "die",
        "Plural": "-en",
        "Satz1_DE": "Eltern haben die Pflicht, ihre Kinder vor Schaden zu bewahren.",
        "Satz1_EN": "Parents have an obligation to protect their children from harm.",
        "Satz2_DE": "Er hatte seine vertraglichen Pflichten erfüllt.",
        "Satz2_EN": "He had met his contractual obligations."
    }
]
```

Please don't output anything else apart from the JSON. The JSON should be valid. The JSON should be an array of objects.

Please add any interesting and relevant information in the "hinweis" field, whatever is connected with a given word, and could help in preparation for the exam.

Use the same structure for the new words and phrases:
"""

In [14]:
cards_to_create_string = """
flattert
belästigt
durchgerostete
Auspuffanlage
"""

# Split by lines and remove empty lines
cards_to_create = [line for line in cards_to_create_string.split("\n") if line != ""]

In [22]:
# We are splitting list on 4 elements chunks
# and then we are creating a list of lists
cards_to_create_chunks = [cards_to_create[x:x+4] for x in range(0, len(cards_to_create), 4)]

for chunk in cards_to_create_chunks[1:]:
    task = task_template + "\n".join(chunk)

    result = openai.ChatCompletion.create(
    model="gpt-4-1106-preview",
    messages=[
            {"role": "user", "content": task}
        ]
    )
    content = result.to_dict_recursive()['choices'][0]["message"]["content"]

    # print(content)
    cleaned_result = clean_content(content)
    # print(cleaned_result)
    new_words = json.loads(cleaned_result)
    print(new_words)

    # Remove None fields
    for word in new_words:
        for key in list(word.keys()):
            if word[key] is None:
                del word[key]
        invoke('addNote', 
        note={
            'deckName': 'Driving License', 
            'modelName': 'Basic (and reversed card)-75aea', 
            'fields': word, 
            'options': {'allowDuplicate': True}, 
            'tags': ['German', 'B1']})


[{'Wort_DE': 'Gütern', 'Wort_EN': 'Goods; Commodities', 'Artikel': 'die', 'Plural': '-', 'Satz1_DE': 'Transporter müssen ihre Ladung von Gütern sicher befestigen.', 'Satz1_EN': 'Cargo vehicles must securely fasten their load of goods.', 'Satz2_DE': 'Diebstahl von Gütern kann zu schwerwiegenden rechtlichen Konsequenzen führen.', 'Satz2_EN': 'Theft of goods can lead to serious legal consequences.', 'hinweis': 'Für die Prüfung ist es wichtig, Verkehrsregeln bezüglich des Transports von Gütern zu kennen.'}, {'Wort_DE': 'einer geschäftsmäßigen oder entgeltlichen Beförderung', 'Wort_EN': 'a commercial or paid transport', 'Satz1_DE': 'Fahrer, die an einer geschäftsmäßigen oder entgeltlichen Beförderung beteiligt sind, müssen über die notwendigen Lizenzen verfügen.', 'Satz1_EN': 'Drivers involved in a commercial or paid transport must have the necessary licenses.', 'Satz2_DE': 'Versicherungsschutz ist besonders wichtig bei einer geschäftsmäßigen oder entgeltlichen Beförderung.', 'Satz2_EN': 'I

In [5]:
result = openai.ChatCompletion.create(
  model="gpt-4-1106-preview",
  messages=[
        {"role": "user", "content": task}
    ]
)
content = result.to_dict_recursive()['choices'][0]["message"]["content"]

In [9]:
# print(content)
cleaned_result = clean_content(content)
# print(cleaned_result)
new_words = json.loads(cleaned_result)
print(new_words)

[{'Wort_DE': 'Wartung', 'Wort_EN': 'Maintenance', 'Artikel': 'die', 'Plural': '-en', 'Satz1_DE': 'Regelmäßige Wartung des Fahrzeugs ist für die Verkehrssicherheit essenziell.', 'Satz1_EN': 'Regular maintenance of the vehicle is essential for road safety.', 'Satz2_DE': 'Während der Wartung werden verschiedene Flüssigkeitsstände und die Bremsen geprüft.', 'Satz2_EN': 'During maintenance, various fluid levels and the brakes are checked.', 'hinweis': 'Eine ordnungsgemäße Wartung des Autos ist notwendig, um sicher am Straßenverkehr teilzunehmen und ist Teil der Prüfungsfragen.'}, {'Wort_DE': 'Radbefestigung', 'Wort_EN': 'Wheel fixation, wheel mounting', 'Artikel': 'die', 'Plural': 'Radbefestigungen', 'Satz1_DE': 'Die Radbefestigung der Vorderräder muss regelmäßig überprüft werden, um Unfälle zu vermeiden.', 'Satz1_EN': 'The wheel fixation of the front wheels must be regularly checked to prevent accidents.', 'Satz2_DE': 'Eine lockere Radbefestigung kann zu schweren Lenkproblemen führen.', 'S

In [11]:
# Remove None fields
for word in new_words:
    for key in list(word.keys()):
        if word[key] is None:
            del word[key]
    invoke('addNote', 
       note={
           'deckName': 'Driving License', 
           'modelName': 'Basic (and reversed card)-75aea', 
           'fields': word, 
           'options': {'allowDuplicate': True}, 
           'tags': ['German', 'B1']})


{'result': 1704640185994, 'error': None}
{'result': 1704640186022, 'error': None}
{'result': 1704640186043, 'error': None}
{'result': 1704640186067, 'error': None}
